In [110]:
import os
import pandas as pd
from pathlib import Path


In [111]:
ls

Dockerfile        artifacts/        requirements.txt  template.py
LICENSE           config/           research/         templates/
README.md         logs/             schema.yaml       venv/
__pycache__/      main.py           setup.py          youtube/
app.py            params.yaml       src/


In [112]:
cd gojuruakshith/

[Errno 2] No such file or directory: 'gojuruakshith/'
/Users/gojuruakshith/YouTube-Fake-Thumbnail-Detector


In [113]:
ls

Dockerfile        artifacts/        requirements.txt  template.py
LICENSE           config/           research/         templates/
README.md         logs/             schema.yaml       venv/
__pycache__/      main.py           setup.py          youtube/
app.py            params.yaml       src/


In [114]:
cd YouTube-Fake-Thumbnail-Detector/

[Errno 2] No such file or directory: 'YouTube-Fake-Thumbnail-Detector/'
/Users/gojuruakshith/YouTube-Fake-Thumbnail-Detector


In [115]:
from dataclasses import dataclass
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [116]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [117]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri= "https://dagshub.com/Gakshith/YouTube-Fake-Thumbnail-Detector.mlflow"

        )

        return model_evaluation_config

In [118]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [119]:
from sklearn.metrics import accuracy_score
import pickle
import dagshub
dagshub.init(repo_owner='Gakshith', repo_name='YouTube-Fake-Thumbnail-Detector', mlflow=True)

import mlflow

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    def log_into_mlflow(self):
          test_data = pd.read_csv(self.config.test_data_path)
          with open(self.config.model_path, "rb") as file:
               model = pickle.load(file)
          X_test = test_data["headline"].astype(str)
          y_test = test_data[self.config.target_column].values.ravel()
          mlflow.set_tracking_uri(self.config.mlflow_uri)

          with mlflow.start_run():
               y_pred = model.predict(X_test)
               (rmse, mae, r2) = self.eval_metrics(y_test, y_pred)
               scores = {"rmse": rmse, "mae": mae, "r2": r2}
               save_json(path=Path(self.config.metric_file_name), data=scores)
               accuracy = accuracy_score(y_test, y_pred)
               # mlflow.log_params(self.config.all_params)
               mlflow.log_metric("rmse", rmse)
               mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
               mlflow.sklearn.log_model(model, "model")

[2026-01-07 22:35:39,378: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/Gakshith/YouTube-Fake-Thumbnail-Detector "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Gakshith/YouTube-Fake-Thumbnail-Detector"

[2026-01-07 22:35:39,383: INFO: helpers: Initialized MLflow to track repo "Gakshith/YouTube-Fake-Thumbnail-Detector"]


Repository Gakshith/YouTube-Fake-Thumbnail-Detector initialized!

[2026-01-07 22:35:39,384: INFO: helpers: Repository Gakshith/YouTube-Fake-Thumbnail-Detector initialized!]


In [120]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2026-01-07 22:35:39,461: INFO: common: YAML file loaded successfully: config/config.yaml]
[2026-01-07 22:35:39,464: INFO: common: YAML file loaded successfully: params.yaml]
[2026-01-07 22:35:39,468: INFO: common: YAML file loaded successfully: schema.yaml]
[2026-01-07 22:35:39,471: INFO: common: Created directory at: artifacts]
[2026-01-07 22:35:39,471: INFO: common: Created directory at: artifacts/model_evaluation]
[2026-01-07 22:35:39,973: INFO: common: JSON file saved at: artifacts/model_evaluation/metrics.json]


2026/01/07 22:35:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run lyrical-wren-116 at: https://dagshub.com/Gakshith/YouTube-Fake-Thumbnail-Detector.mlflow/#/experiments/0/runs/36da5bf0686f487d8ba3eeae48caaaf6
🧪 View experiment at: https://dagshub.com/Gakshith/YouTube-Fake-Thumbnail-Detector.mlflow/#/experiments/0
